In [315]:
import requests
import pandas as pd

class ScriptData:
    def __init__(self, api_key):
        self.api_key = api_key
        self.intraday_data = {}
    
    def fetch_intraday_data(self, script):
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={script}&interval=5min&apikey={self.api_key}"
        response = requests.get(url)
        self.intraday_data[script] = response.json()
    
    def convert_intraday_data(self, script):
        raw_data = self.intraday_data[script]["Time Series (5min)"]
        data = []
        for timestamp, values in raw_data.items():
            data.append([pd.Timestamp(timestamp), float(values["1. open"]), float(values["2. high"]), float(values["3. low"]), float(values["4. close"]), int(values["5. volume"])])
        self.intraday_data[script] = pd.DataFrame(data, columns=["timestamp", "open", "high", "low", "close", "volume"])
    
    def __getitem__(self, script):
        return self.intraday_data[script]
    
    def __setitem__(self, script, df):
        self.intraday_data[script] = df
    
    def __contains__(self, script):
        return script in self.intraday_data


In [316]:
script_data=ScriptData('7EDMDEL5JTTNFWOI')

In [317]:
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2023-02-10 20:00:00,94.590,94.6900,94.5800,94.6900,3426
1,2023-02-10 19:55:00,94.600,94.6200,94.6000,94.6000,3043
2,2023-02-10 19:50:00,94.590,94.6900,94.5900,94.6000,2719
3,2023-02-10 19:45:00,94.570,94.6800,94.5700,94.6300,4798
4,2023-02-10 19:40:00,94.580,94.6300,94.5700,94.5700,5282
...,...,...,...,...,...,...
95,2023-02-10 12:05:00,94.600,94.6513,94.2800,94.3785,1227425
96,2023-02-10 12:00:00,94.960,94.9600,94.5800,94.6050,704617
97,2023-02-10 11:55:00,95.055,95.1000,94.9050,94.9600,355965
98,2023-02-10 11:50:00,94.990,95.0900,94.9600,95.0550,401343


In [318]:
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')
script_data['AAPL']

,timestamp,open,high,low,close,volume
0,2023-02-10 20:00:00,151.200,151.2600,151.2000,151.2600,10821
1,2023-02-10 19:55:00,151.140,151.2000,151.1400,151.2000,2397
2,2023-02-10 19:50:00,151.070,151.0900,151.0700,151.0900,1147
3,2023-02-10 19:45:00,151.040,151.0500,151.0400,151.0500,1558
4,2023-02-10 19:40:00,151.010,151.0400,151.0100,151.0100,631
...,...,...,...,...,...,...
95,2023-02-10 11:50:00,150.110,150.1600,149.7800,149.9600,517929
96,2023-02-10 11:45:00,150.210,150.3000,150.0800,150.1100,291976
97,2023-02-10 11:40:00,150.110,150.2800,150.1000,150.2200,351659
98,2023-02-10 11:35:00,150.300,150.3499,150.0150,150.1200,406347


In [319]:
'GOOGL' in script_data

True

In [320]:
'AAPL' in script_data

True

In [321]:
'NVDA' in script_data

False

In [322]:
def indicator1(df, timeperiod):
    result = df.copy()
    result["indicator"] = result["close"].rolling(window=timeperiod).mean()
    return result[["timestamp", "indicator"]]


In [323]:
indicator1(script_data['GOOGL'], timeperiod=5)

,timestamp,indicator
0,2023-02-10 20:00:00,NaN
1,2023-02-10 19:55:00,NaN
2,2023-02-10 19:50:00,NaN
3,2023-02-10 19:45:00,NaN
4,2023-02-10 19:40:00,94.6180
...,...,...
95,2023-02-10 12:05:00,94.4366
96,2023-02-10 12:00:00,94.4466
97,2023-02-10 11:55:00,94.5326
98,2023-02-10 11:50:00,94.6576


In [324]:
indicator1(script_data['AAPL'], timeperiod=5)

,timestamp,indicator
0,2023-02-10 20:00:00,NaN
1,2023-02-10 19:55:00,NaN
2,2023-02-10 19:50:00,NaN
3,2023-02-10 19:45:00,NaN
4,2023-02-10 19:40:00,151.12200
...,...,...
95,2023-02-10 11:50:00,149.80018
96,2023-02-10 11:45:00,149.89418
97,2023-02-10 11:40:00,150.02000
98,2023-02-10 11:35:00,150.08000


In [371]:
class Strategy:
    def __init__(self, script):
        self.script = script
        self.script_data = ScriptData('7EDMDEL5JTTNFWOI')
        self.df = None
        self.indicator_data = None
        self.signals = None
        
    def get_script_data(self):
        self.df = self.script_data.fetch_intraday_data(self.script)
        self.df = self.script_data.convert_intraday_data(self.script)
        self.df = self.script_data[self.script]
        
    def get_signals(self):
        if self.df is None:
            self.get_script_data()
            
        self.indicator_data = indicator1(self.df, 5)
        self.signals = pd.DataFrame({"timestamp": self.df["timestamp"]})
        self.signals["signal"] = "NO_SIGNAL"
        
        buy_indices = (self.indicator_data["indicator"] > self.df["close"]) & (self.indicator_data["indicator"].shift(1) <= self.df["close"].shift(1))
        sell_indices = (self.indicator_data["indicator"] < self.df["close"]) & (self.indicator_data["indicator"].shift(1) >= self.df["close"].shift(1))
        self.signals.loc[buy_indices, "signal"] = "BUY"
        self.signals.loc[sell_indices, "signal"] = "SELL"
        
        print(self.signals[self.signals["signal"] != "NO_SIGNAL"])



In [359]:
strategy = Strategy('NVDA')

In [360]:
strategy.get_script_data()

In [361]:
strategy.get_signals()

             timestamp signal
6  2023-02-10 19:30:00   SELL
7  2023-02-10 19:25:00    BUY
8  2023-02-10 19:20:00   SELL
12 2023-02-10 19:00:00    BUY
13 2023-02-10 18:55:00   SELL
15 2023-02-10 18:45:00    BUY
16 2023-02-10 18:40:00   SELL
17 2023-02-10 18:35:00    BUY
21 2023-02-10 18:15:00   SELL
22 2023-02-10 18:10:00    BUY
26 2023-02-10 17:50:00   SELL
29 2023-02-10 17:35:00    BUY
36 2023-02-10 17:00:00   SELL
38 2023-02-10 16:50:00    BUY
41 2023-02-10 16:35:00   SELL
44 2023-02-10 16:20:00    BUY
47 2023-02-10 16:05:00   SELL
50 2023-02-10 15:50:00    BUY
57 2023-02-10 15:15:00   SELL
59 2023-02-10 15:05:00    BUY
64 2023-02-10 14:40:00   SELL
65 2023-02-10 14:35:00    BUY
68 2023-02-10 14:20:00   SELL
72 2023-02-10 14:00:00    BUY
73 2023-02-10 13:55:00   SELL
81 2023-02-10 13:15:00    BUY
91 2023-02-10 12:25:00   SELL
92 2023-02-10 12:20:00    BUY
93 2023-02-10 12:15:00   SELL


In [383]:
import plotly.graph_objs as go

def plot_candlestick_chart(script):
    script_data = ScriptData('7EDMDEL5JTTNFWOI')
    df = script_data.fetch_intraday_data(script)
    df = script_data.convert_intraday_data(script)
    df = script_data[script]
    indicator_data = indicator1(df, 5)

    fig = go.Figure(data=[go.Candlestick(x=df['timestamp'],
                                          open=df['open'],
                                          high=df['high'],
                                          low=df['low'],
                                          close=df['close']),
                       go.Scatter(x=indicator_data['timestamp'],
                                  y=indicator_data['indicator'],
                                  mode='lines',
                                  name='Indicator Data')])

    fig.update_layout(title='Candlestick Chart of Price Data with Indicator')
    fig.show()


In [384]:
plot_candlestick_chart('AAPL')